In [1]:
import os
import pymongo
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

MONGO_DB_USER = os.environ["MONGO_DB_USER"]
MONGO_DB_PASSWORD = os.environ["MONGO_DB_PASSWORD"]
MONGO_DB_HOST = os.environ["MONGO_DB_HOST"]
MONGO_DB_PORT = os.environ["MONGO_DB_PORT"]

client = pymongo.MongoClient(
    f"mongodb://{MONGO_DB_USER}:{MONGO_DB_PASSWORD}@{MONGO_DB_HOST}:{MONGO_DB_PORT}/"
)

silver_db = client.silver
silver_events = silver_db.events
silver_games = silver_db.games

In [2]:
# Find out how many times each player won

c = silver_events.aggregate(
    [
        {"$project": {"max": {"$objectToArray": {"$max": "$results"}}}},
        {"$project": {"results": {"$arrayElemAt": ["$max.k", 0]}}},
    ]
)

winners = {}

for i in c:
    winners.setdefault(i["results"], 0)
    winners[i["results"]] += 1

winners

{'VovAn1991': 1,
 'Hafez_Bakr': 1,
 'Philofive': 1,
 'silverback96': 1,
 'GulamaliRises': 1,
 'Dlugy': 3,
 'MITerryble': 2,
 'MattyDPerrine': 1,
 'Canonkasparov': 1,
 'Yaacovn': 2,
 'AlexanderL': 2,
 'kpio91': 1,
 'Tal-Baronfew': 1,
 'spectralsoul': 1,
 'SerGo055': 1,
 'Gergo85': 1,
 'GrigorGrigorov': 2,
 'yurisolo': 1,
 'Hikaru': 4,
 'stefanodinn': 1,
 'Quiet_Desperation': 1,
 'ZEN-22': 1,
 'GM_Huschenbeth': 1,
 'Paralinch': 1,
 'Boryboy': 1,
 'vovkcoach': 1,
 'jlandaw': 1,
 'jcibarra': 2,
 'Newlight1': 1,
 'Tonychess32': 1,
 'BCNUM8': 1,
 'Journey_to_GrandMaster_YT': 1,
 'lionheart95': 1,
 'psv98': 1,
 'VMikhalevski': 1,
 'NTBT_DHS': 1,
 'KuzubovYuriy': 1,
 'JohanSalomon': 1,
 'CemilCan': 1,
 'Kouzari': 1,
 'severomorskij': 1,
 'LyonBeast': 1,
 'chito89': 1,
 'biveklama': 1,
 'rst2003': 1,
 'fastestmindalive': 1,
 'wnstop': 1,
 'RitvarsReimanis': 1,
 'GianmarcoVaR': 1,
 'GMMarcinDziuba': 1,
 'grzechu96': 1,
 'champ2005': 2,
 'Vojtastr': 1,
 'Aygehovit1992': 1,
 'Markov_Mikhail': 1,
 

In [3]:
sorted(winners.items(), key=lambda x: x[1])[-2:]

[('Carlsen, Magnus', 26), ('Nakamura, Hikaru', 38)]

In [4]:
upsets = (
    silver_games.find({"WhiteElo": {"$gt": 1000}, "BlackElo": {"$gt": 1000}})
    .limit(10)
    .sort({"Upset": 1})
)

pd.DataFrame(upsets)[["Event", "Upset", "White", "WhiteElo", "Black", "BlackElo"]]

,Event,Upset,White,WhiteElo,Black,BlackElo
0,Titled Tuesday Blitz December 20 Early 2022,-1619.0,Susal_De_Silva,2808.0,"Kazgildy, Fariza",1189.0
1,Titled Tuesday Blitz October 18 Late 2022,-1546.0,"Eisa Mohmood, Alkhoori",1245.0,"Bordi, Kevin",2791.0
2,Titled Tuesday Blitz December 20 Early 2022,-1536.0,Kopylov_Alex,2790.0,"Mone, Ved",1254.0
3,Titled Tuesday Blitz December 20 Early 2022,-1463.0,The_Machine04,2652.0,"Kazgildy, Fariza",1189.0
4,Titled Tuesday Blitz April 26 Early 2022,-1414.0,"Bashirli, Saadat",1173.0,"Bachmann, Axel",2587.0
5,Titled Tuesday Blitz December 20 Early 2022,-1393.0,"Benamar, Benious",1136.0,lemayvega,2529.0
6,Titled Tuesday Blitz December 20 Early 2022,-1350.0,CDOEspiao,2486.0,"Benamar, Benious",1136.0
7,*** Titled Tuesday Blitz,-1328.0,Markov_Mikhail,2665.0,xpto15,1337.0
8,Titled Tuesday Blitz December 20 Early 2022,-1328.0,"Castro Andrade, Seirbert Cristian",1364.0,"Xiong, Jeffery",2692.0
9,Titled Tuesday Blitz December 20 Early 2022,-1311.0,"Yildiz, Ayse Emek",1702.0,OparinGrigoriy,3013.0


In [5]:
upsets = (
    silver_games.find(
        {
            "White": {"$in": ["Carlsen, Magnus"]},
            "WhiteElo": {"$gt": 1000},
            "BlackElo": {"$gt": 1000},
        }
    )
    .limit(3)
    .sort({"Upset": 1})
)

pd.DataFrame(upsets)[["Event", "Upset", "White", "WhiteElo", "Black", "BlackElo"]]

,Event,Upset,White,WhiteElo,Black,BlackElo
0,Titled Tuesday Blitz March 21 Early 2023,-475.0,"Carlsen, Magnus",2852.0,"Tillyaev, Ulugbek",2377.0
1,Titled Tuesday Blitz May 9 Late 2023,-391.0,"Carlsen, Magnus",2852.0,"Makarian, Rudik",2461.0
2,Titled Tuesday Blitz July 25 Late 2023,-355.0,"Carlsen, Magnus",3256.0,"Dardha, Daniel",2901.0


In [6]:
upsets = (
    silver_games.find(
        {
            "White": {"$in": ["Nakamura, Hikaru"]},
            "WhiteElo": {"$gt": 1000},
            "BlackElo": {"$gt": 1000},
        }
    )
    .limit(3)
    .sort({"Upset": 1})
)

pd.DataFrame(upsets)[["Event", "Upset", "White", "WhiteElo", "Black", "BlackElo"]]

,Event,Upset,White,WhiteElo,Black,BlackElo
0,Titled Tuesday Blitz August 15 Early 2023,-1180.0,"Nakamura, Hikaru",3255.0,"Nugumanov, Bakhtiyar",2075.0
1,Titled Tuesday Blitz May 10 Early 2022,-1054.0,"Nakamura, Hikaru",2850.0,"Aslanov, Umid",1796.0
2,Titled Tuesday Blitz June 6 Late 2023,-759.0,"Nakamura, Hikaru",2874.0,"Hajiyev, Kanan",2115.0
